<a href="https://colab.research.google.com/github/kwanda2426/projects/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries**

In [ ]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import plotly.express as px

from statsmodels.graphics.correlation import plot_corr
from scipy.stats import skew
from scipy.stats import kurtosis
import statistics
import missingno as msno

#Feature engineering, selection and Model training
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder


#ignoring warnings
import warnings
warnings.filterwarnings('ignore')

#making sure that we can see all rows and cols
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from tabulate import tabulate


# **Importing Data**

In [ ]:
# Reading data
file_path = "https://raw.githubusercontent.com/kwanda2426/projects/main/insurance.csv"
df = pd.read_csv(file_path)


In [ ]:
# Showing first 5 lines of data
df.head()


In [ ]:
#data columns
df.columns

In [ ]:
# Rows and Columns of data
print('Data has {} rows and {} Columns'.format(df.shape[0],df.shape[1]))
print('')
print('Data has the following columns:')
for i in df.columns:
  print(i)

### Data Type of Columns

In [ ]:
df.info()

Data set has no missing values, with a mixture of both categorical and numerical features.

## Describing the variables

In [ ]:
#describing numerical columns
df.describe()

In [ ]:
#describing categorical variables
cat_cols = df.select_dtypes(include=["object", "category"])
for i in cat_cols.columns:
  a = df[i].unique()
  print('unique categories for '+ i.upper() + ' is(are) :{}'.format(a))
  print('')


In [ ]:
#Defining the variables
variables = {
    "Variable": ["age", "sex", "bmi", "children", "smoker", "region", "charges"],
    "Description": [
        "Age of the insured individual (between 18 and 64).",
        "Gender of the insured person (male or female).",
        "Body Mass Index, a measure of body fat based on height and weight (between 15 and 54).",
        "Number of dependent children covered by the insurance plan (between 0 and 5).",
        "Smoking status of the individual (yes or no).",
        "Residential area (northeast, northwest, southeast, southwest).",
        "Final insurance cost billed to the individual (between 1121 and 63770)."
    ]
}
# Create the DataFrame
var_desc = pd.DataFrame(variables)
var_desc.reset_index(drop=True, inplace=True)

# Display the DataFrame
print(tabulate(var_desc, headers='keys', tablefmt='pretty'))


The categorical variables will need to be encoded if they will be used in building the model.

# **Exploratory Data Analysis**

This method is meant to uncover the underlying structure of a data set and is important for a company because it exposes trends, patterns, and relationships that are not readily apparent.

## **Correlation**
This is meant to discover :
- Relationship between features and target variable.
- Multicollinearity - where features are highly correlated to each other. This guides whether to combine features or not as this means that features carry the same information.
- Most informative features. This helps to (simplify the model, reduce noise and improve predictive accuracy) by focusing on informative features only.
- Patterns such as if one feature increases, so does another OR One decreases , the other increases.
- +1 shows a strong positive correlation and -1 shows a strong negative correlation.


In [ ]:
# Compute correlation matrix
df_corr = df.copy()
corr_matrix = df_corr.corr(numeric_only=True)

# Create heatmap
fig = px.imshow(
    corr_matrix,
    text_auto = True,
    color_continuous_scale = 'RdBu',
    zmin = -1, zmax = 1,
    title = 'Feature Correlation Heatmap'
)

fig.update_layout(
    width=800,
    height=600,
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.show()


- There's a positive strong correlation between age and charges. This means that as the age increases so do insurances.
- There's a strong correlation between BMI and Charges, which is expected because BMI is a function of Age.
- Positive correlation between BMI and age is not strong enough to drop one of these features even though they are related. Model performance will guide if one feature must be dropped.

## **Feature Distribution**
This is meant to discover :
- Relationship between features and target variable. This should help see if a feature that was not caught by correlation because of datatype(especially categorical) has a relationship with the target variable. This helps in encoding categorical features. For an example, with the region in our dataset. It could be that the feature does not affect charges at all, or some regions have more impact than others.

###**Gender(sex) Distribution**

In [ ]:
#countplot of sex (between males and females)
df_1 = df.copy()
sns.countplot(x = 'sex', data = df_1, palette = "hls")
plt.title("Distribution of sex");

These two groups are approximately well represented in our dataset, hence there's expected bias in our model predictions with regards to this feature.

In [ ]:
#distribution of charges per age group, by gender
df_1 = df.copy()
plt.figure(figsize = (12,8))
sns.scatterplot(data = df_1, x ="age", y = "charges", hue = "sex", alpha=0.8, sizes=(40,200))
plt.title("Charges vs Age by Sex")
plt.show()

Here it is clear that it does not matter whether someone is male or female when it comes to insurance charges. An upward trend is seen between charges and Age.

In [ ]:
# Group by gender and smoker status, then calculate average charges
df_1 = df.copy()
df_temp = df_1.groupby(['sex', 'smoker'], as_index = False)['charges'].mean()

# Stacked bar chart
fig = px.bar(df_temp,
             x = 'sex',
             y = 'charges',
             color = 'smoker',
             title = 'Average Charges per Gender (Stacked by Smoking Status)',
             labels = {'charges': 'Average Charges', 'sex': 'Gender', 'smoker': 'Smoking Status'},
             text = 'charges')

fig.update_traces(texttemplate = '%{text:.2f}', textposition='inside')
fig.update_layout(barmode = 'stack', uniformtext_minsize = 8, uniformtext_mode = 'hide')
fig.show()

On Average Male charges are slightly above Average Female, but not by a significant amount. This view conform to what is shown above that Gender(sex) does have a significant impact on insurance charges. This feature might not be useful to predict charges, but dropping it should be guided by model performance.

###**Region Distribution**

In [ ]:
#countplot of Region
df_1 = df.copy()
sns.countplot(x = 'region', data = df_1, palette = "hls")
plt.title("Distribution of Region");

Looks like all regions are fairly represented in the dataset. Should this feature be included in model building, there would not be a bias expected.

In [ ]:
#distribution of charges per age group, by region
df_1 = df.copy()
plt.figure(figsize = (12,8))
sns.scatterplot(data = df_1, x = "age", y = "charges", hue = "region", alpha=0.8, sizes=(40,200))
plt.title("Charges vs Age by Region")
plt.show()

This views shows that all these regions are fairly represented even with age. The trend that is visible is the increase in charges as the age increases.

In [ ]:
# Group by region and smoker status, then calculate average charges
df_1 = df.copy()
df_temp = df_1.groupby(['region', 'smoker'], as_index = False)['charges'].mean()

# Stacked bar chart
fig = px.bar(df_temp,
             x = 'region',
             y = 'charges',
             color = 'smoker',
             title = 'Average Charges per Region (Stacked by Smoking Status)',
             labels = {'charges': 'Average Charges', 'region': 'Region', 'smoker': 'Smoking Status'},
             text = 'charges')

fig.update_traces(texttemplate = '%{text:.2f}', textposition='inside')
fig.update_layout(barmode = 'stack', uniformtext_minsize = 8, uniformtext_mode = 'hide')
fig.show()

- It is clear that it does matter significantly in which region one comes from when it comes to charges, but the smoking status does. It is clear how significant the charges increase when someone is a smoker.

###**Age Distribution**

In [ ]:
df_1 = df.copy()
features = 'age'
df_1[features].hist(figsize = (12,10),  color = '#000354');

Most data points are of the age 20. The other age groups are fairly represented as well.

In [ ]:
#distribution of charges per age group, by smoking status
df_1 = df.copy()
plt.figure(figsize = (12,8))
sns.scatterplot(data = df_1, x = "age", y = "charges", hue = "smoker", alpha=0.8, sizes=(40,200))
plt.title("Charges vs Age by Smoking Status")
plt.show()

- There is a clear trend that irregardless of age, smokers end up paying more charges.
- Where charges are less than 10k - This comprises of non-smokers for all age groups.
- Whether charges are between 10k and 30k, both non-smokers and smokers are fairly represented and this is where we have average charges per age group.
- Where charges are greater than 30k - This group consists of virtually smokers.

In [ ]:
# Create DataFrame
df_1 = df.copy()

# Set Seaborn style
sns.set(style="whitegrid")

# Create line plot
plt.figure(figsize=(10, 6))
sns.lineplot(x = 'age', y = 'charges', data = df_1, marker='o', color='teal')

# Add labels and title
plt.title('Charges vs Age', fontsize=16)
plt.xlabel('Age', fontsize = 12)
plt.ylabel('Charges', fontsize=12)

# Show plot
plt.tight_layout()
plt.show()

Age definitely does have an impact in charges. As the age increases so do charges. Older people are charge a lot.

In [ ]:
# Average Monthly charges vs age
df_1 = df.copy()
df_temp = df_1.groupby('age')['charges'].mean().reset_index()
# Plot
plt.figure(figsize=(10, 6))
sns.regplot(data = df_temp, x = 'age', y = 'charges', scatter = True, color = 'skyblue', line_kws = {'color': 'red'})
plt.title('Average Monthly Charges per Age')
plt.xlabel('Age')
plt.ylabel('Average Monthly Charges')
plt.grid(True)
plt.tight_layout()
plt.show()

Trend is going up between charges and age. This shows a strong positive correlation between these two features.

# **Feature Selection and Engineering**


- This dataset does not have a lot of features(6). It is better to build the model first and see performance before dropping features.
- In terms of feature engineering, the categorical variables are converted to numerical variables using label encoding.



In [ ]:
# encoding categorical features
def Label_Encoder(df_1):
  columnsToEncode = list(df.select_dtypes(include=['category','object']))
  le = LabelEncoder()
  for feature in columnsToEncode:
    df[feature] = le.fit_transform(df[feature])
  return df

# final dataframe
df_1 = df.copy()
df_final = Label_Encoder(df_1)

In [ ]:
df_final.head()

# **Model Building and Testing**

In [112]:
# splitting dataset into training and testing
df_data = df_final.copy()
X = df_data.drop(['charges'], axis = 1)
Y = df_data['charges']

# 80% train and 20% test -- random state = 42, ensure the reproducibility of the data split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [113]:
# initialise model
model = LinearRegression()

#train model
model.fit(x_train, y_train)

LinearRegression()

# **Model Evaluation**